In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\jaswe\\OneDrive\\Desktop\\opt\\Projects\\Chicken_Disease_Classification_Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [5]:
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constant import *


In [10]:
class PrepareCallbackConfigurationManager:
    def __init__(
            self, config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_call_backs
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class prepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
            monitor="val_loss",    
            mode="auto",           
            save_weights_only=False
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callback,
            self._create_ckpt_callbacks
        ]

In [15]:
try:
    config = PrepareCallbackConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = prepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e


[2025-04-27 23:02:51,335: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-27 23:02:51,338: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-27 23:02:51,339: INFO: common: created directory at: artifacts]
[2025-04-27 23:02:51,341: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-04-27 23:02:51,342: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
